# Simulation of M9 medium with and without Casamino acids
Author: Famke Baeuerle

In [1]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd
import numpy as np

In [2]:
def enhanced_growth(model, base_medium, new_metabs):
    local_medium = base_medium
    with model:
        medium = model.medium
        for met in new_metabs:
            local_medium[met] = 10.0
        model.medium = local_medium
        sol = model.optimize()
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate with additives: ' + str(round(base_growth, 4)))
    print('Doubling time with additives: ' + str(round(dt, 2)) + ' min')

def find_additives(model, base_medium):
    with model:
        medium = model.medium
        model.medium = base_medium
        sol = model.optimize()      
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate without additives: ' + str(round(base_growth, 4)))
    print('Doubling time without additives: ' + str(round(dt, 2)) + ' min')
        
    enhancement = {}
    for ex in list(model.exchanges):
        if ex.id not in base_medium.keys():
            with model:
                medium = model.medium
                base_medium.update({ex.id:10.0})
                model.medium = base_medium
                sol = model.optimize()
                if sol.objective_value > base_growth:
                    enhancement[ex.id] = sol.objective_value - base_growth
                base_medium.pop(ex.id)
    
    enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff']).sort_values(by=['diff'], ascending=False)
    
    return enh

In [43]:
CasA = rg.growth.load_all_media_from_db('../../../refinegems/data/media_db.csv')[9]['BiGG_EX'].to_list()
M9 = rg.growth.load_all_media_from_db('../../../refinegems/data/media_db.csv')[2]['BiGG_EX'].to_list()
CasA_med = {i: 10.0 for i in CasA}
M9_med = {i: 10.0 for i in M9}
if (M9_med['EX_o2_e'] == 10.0):
    M9_med['EX_o2_e'] = 20.0
M9_med.pop('EX_ni2_e') # missing in the models

# from simulation with refinegems
M9_med['EX_ala_B_e'] = 10.0
M9_med['EX_cgly_e'] = 10.0

In [31]:
model = rg.load.load_model_cobra('../../models/Cstr_1197.xml')

In [32]:
find_additives(model, M9_med)

Growth rate without additives: 0.5531
Doubling time without additives: 75.19 min


,exchange,diff
66,EX_malthp_e,1.968833e+00
93,EX_sucr_e,4.721784e-01
102,EX_tre_e,4.721784e-01
65,EX_malt_e,4.721784e-01
61,EX_lcts_e,3.851246e-01
...,...,...
91,EX_stfrnB_e,3.652634e-14
104,EX_tsul_e,3.586020e-14
81,EX_ptrc_e,1.498801e-14
99,EX_tcynt_e,6.217249e-15


In [33]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.639
Doubling time with additives: 65.08 min


In [34]:
model = rg.load.load_model_cobra('../../models/Cstr_TS.xml')

In [35]:
find_additives(model, M9_med)

Growth rate without additives: 0.6368
Doubling time without additives: 65.31 min


,exchange,diff
89,EX_malthp_e,2.319662e+00
129,EX_tre_e,5.458029e-01
88,EX_malt_e,5.458029e-01
120,EX_sucr_e,5.458029e-01
87,EX_lcts_e,4.775775e-01
...,...,...
102,EX_progly_e,1.332268e-15
101,EX_pro__L_e,1.332268e-15
99,EX_pime_e,1.332268e-15
98,EX_pheme_e,1.332268e-15


In [36]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.6368
Doubling time with additives: 65.31 min


In [37]:
model = rg.load.load_model_cobra('../../models/Cstr_1115.xml')

In [38]:
find_additives(model, M9_med)

Growth rate without additives: 0.7652
Doubling time without additives: 54.35 min


,exchange,diff
115,EX_malthp_e,2.295648e+00
156,EX_sucr_e,5.401524e-01
165,EX_tre_e,5.401524e-01
114,EX_malt_e,5.401524e-01
108,EX_lcts_e,4.726333e-01
...,...,...
58,EX_dca_e,7.080669e-12
101,EX_ind3ac_e,7.076673e-12
151,EX_so3_e,7.060352e-12
65,EX_fum_e,7.032042e-12


In [39]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.7652
Doubling time with additives: 54.35 min


In [44]:
model = rg.load.load_model_cobra('../../models/Cstr_1116.xml')
#M9_med['EX_nmn_e'] = 10.0

In [45]:
find_additives(model, M9_med)

/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_12300/2573724546.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60


Growth rate without additives: 0.0
Doubling time without additives: inf min


,exchange,diff
0,EX_nmn_e,5.669840e-01
27,EX_ser__L_e,3.983934e-12
29,EX_skm_e,3.983934e-12
30,EX_so3_e,3.983934e-12
31,EX_spmd_e,3.983934e-12
32,EX_stfrnA_e,3.983934e-12
33,EX_stfrnB_e,3.983934e-12
34,EX_succ_e,3.983934e-12
35,EX_sucr_e,3.983934e-12
36,EX_tag160_e,3.983934e-12


In [46]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.0
Doubling time with additives: 10439135720423.6 min
